In [27]:
import os
import nmslib
import numpy as np

# towhee is a library that converts images into vectors
from towhee import AutoPipes

# HNSW initialization with Euclidean operations
index = nmslib.init(method = 'hnsw', space = 'l2')
p = AutoPipes.pipeline('image-embedding')

path = './JPEGImages'
# get a list of all the images in the JPEGImages directory
dir_list = os.listdir(path)

length = len(dir_list)

for i in range(0, length):
    print(f'{i}/{length}')
    
    # JPEGImages/currentFile
    filePath = f'{path}/{dir_list[i]}'
    res = p(filePath)
    
    # adds a node into HNSW
    index.addDataPoint(i, res.get())
index.createIndex({ 'post': 2 })

# True means image and database is saved
index.saveIndex('imageDB', True)

# Test
query = p('./JPEGImages/2012_004330.jpg').get()

k = 5

ids, distances = index.knnQuery(query, k=k)

# Print the results
print("Query Point:", query)
print("Nearest Neighbors:")
for i, idx in enumerate(ids):
    print("Index:", idx, "Distance:", distances[i])

0/17125
1/17125
2/17125
3/17125
4/17125
5/17125
6/17125
7/17125
8/17125
9/17125
10/17125
11/17125
12/17125
13/17125
14/17125
15/17125
16/17125
17/17125
18/17125
19/17125
20/17125
21/17125
22/17125
23/17125
24/17125
25/17125
26/17125
27/17125
28/17125
29/17125
30/17125
31/17125
32/17125
33/17125
34/17125
35/17125
36/17125
37/17125
38/17125
39/17125
40/17125
41/17125
42/17125
43/17125
44/17125
45/17125
46/17125
47/17125
48/17125
49/17125
50/17125
51/17125
52/17125
53/17125
54/17125
55/17125
56/17125
57/17125
58/17125
59/17125
60/17125
61/17125
62/17125
63/17125
64/17125
65/17125
66/17125
67/17125
68/17125
69/17125
70/17125
71/17125
72/17125
73/17125
74/17125
75/17125
76/17125
77/17125
78/17125
79/17125
80/17125
81/17125
82/17125
83/17125
84/17125
85/17125
86/17125
87/17125
88/17125
89/17125
90/17125
91/17125
92/17125
93/17125
94/17125
95/17125
96/17125
97/17125
98/17125
99/17125
100/17125
101/17125
102/17125
103/17125
104/17125
105/17125
106/17125
107/17125
108/17125
109/17125
110/17125


832/17125
833/17125
834/17125
835/17125
836/17125
837/17125
838/17125
839/17125
840/17125
841/17125
842/17125
843/17125
844/17125
845/17125
846/17125
847/17125
848/17125
849/17125
850/17125
851/17125
852/17125
853/17125
854/17125
855/17125
856/17125
857/17125
858/17125
859/17125
860/17125
861/17125
862/17125
863/17125
864/17125
865/17125
866/17125
867/17125
868/17125
869/17125
870/17125
871/17125
872/17125
873/17125
874/17125
875/17125
876/17125
877/17125
878/17125
879/17125
880/17125
881/17125
882/17125
883/17125
884/17125
885/17125
886/17125
887/17125
888/17125
889/17125
890/17125
891/17125
892/17125
893/17125
894/17125
895/17125
896/17125
897/17125
898/17125
899/17125
900/17125
901/17125
902/17125
903/17125
904/17125
905/17125
906/17125
907/17125
908/17125
909/17125
910/17125
911/17125
912/17125
913/17125
914/17125
915/17125
916/17125
917/17125
918/17125
919/17125
920/17125
921/17125
922/17125
923/17125
924/17125
925/17125
926/17125
927/17125
928/17125
929/17125
930/17125
931/17125


1595/17125
1596/17125
1597/17125
1598/17125
1599/17125
1600/17125
1601/17125
1602/17125
1603/17125
1604/17125
1605/17125
1606/17125
1607/17125
1608/17125
1609/17125
1610/17125
1611/17125
1612/17125
1613/17125
1614/17125
1615/17125
1616/17125
1617/17125
1618/17125
1619/17125
1620/17125
1621/17125
1622/17125
1623/17125
1624/17125
1625/17125
1626/17125
1627/17125
1628/17125
1629/17125
1630/17125
1631/17125
1632/17125
1633/17125
1634/17125
1635/17125
1636/17125
1637/17125
1638/17125
1639/17125
1640/17125
1641/17125
1642/17125
1643/17125
1644/17125
1645/17125
1646/17125
1647/17125
1648/17125
1649/17125
1650/17125
1651/17125
1652/17125
1653/17125
1654/17125
1655/17125
1656/17125
1657/17125
1658/17125
1659/17125
1660/17125
1661/17125
1662/17125
1663/17125
1664/17125
1665/17125
1666/17125
1667/17125
1668/17125
1669/17125
1670/17125
1671/17125
1672/17125
1673/17125
1674/17125
1675/17125
1676/17125
1677/17125
1678/17125
1679/17125
1680/17125
1681/17125
1682/17125
1683/17125
1684/17125
1685/17125

2343/17125
2344/17125
2345/17125
2346/17125
2347/17125
2348/17125
2349/17125
2350/17125
2351/17125
2352/17125
2353/17125
2354/17125
2355/17125
2356/17125
2357/17125
2358/17125
2359/17125
2360/17125
2361/17125
2362/17125
2363/17125
2364/17125
2365/17125
2366/17125
2367/17125
2368/17125
2369/17125
2370/17125
2371/17125
2372/17125
2373/17125
2374/17125
2375/17125
2376/17125
2377/17125
2378/17125
2379/17125
2380/17125
2381/17125
2382/17125
2383/17125
2384/17125
2385/17125
2386/17125
2387/17125
2388/17125
2389/17125
2390/17125
2391/17125
2392/17125
2393/17125
2394/17125
2395/17125
2396/17125
2397/17125
2398/17125
2399/17125
2400/17125
2401/17125
2402/17125
2403/17125
2404/17125
2405/17125
2406/17125
2407/17125
2408/17125
2409/17125
2410/17125
2411/17125
2412/17125
2413/17125
2414/17125
2415/17125
2416/17125
2417/17125
2418/17125
2419/17125
2420/17125
2421/17125
2422/17125
2423/17125
2424/17125
2425/17125
2426/17125
2427/17125
2428/17125
2429/17125
2430/17125
2431/17125
2432/17125
2433/17125

3093/17125
3094/17125
3095/17125
3096/17125
3097/17125
3098/17125
3099/17125
3100/17125
3101/17125
3102/17125
3103/17125
3104/17125
3105/17125
3106/17125
3107/17125
3108/17125
3109/17125
3110/17125
3111/17125
3112/17125
3113/17125
3114/17125
3115/17125
3116/17125
3117/17125
3118/17125
3119/17125
3120/17125
3121/17125
3122/17125
3123/17125
3124/17125
3125/17125
3126/17125
3127/17125
3128/17125
3129/17125
3130/17125
3131/17125
3132/17125
3133/17125
3134/17125
3135/17125
3136/17125
3137/17125
3138/17125
3139/17125
3140/17125
3141/17125
3142/17125
3143/17125
3144/17125
3145/17125
3146/17125
3147/17125
3148/17125
3149/17125
3150/17125
3151/17125
3152/17125
3153/17125
3154/17125
3155/17125
3156/17125
3157/17125
3158/17125
3159/17125
3160/17125
3161/17125
3162/17125
3163/17125
3164/17125
3165/17125
3166/17125
3167/17125
3168/17125
3169/17125
3170/17125
3171/17125
3172/17125
3173/17125
3174/17125
3175/17125
3176/17125
3177/17125
3178/17125
3179/17125
3180/17125
3181/17125
3182/17125
3183/17125

...

13951/17125
13952/17125
13953/17125
13954/17125
13955/17125
13956/17125
13957/17125
13958/17125
13959/17125
13960/17125
13961/17125
13962/17125
13963/17125
13964/17125
13965/17125
13966/17125
13967/17125
13968/17125
13969/17125
13970/17125
13971/17125
13972/17125
13973/17125
13974/17125
13975/17125
13976/17125
13977/17125
13978/17125
13979/17125
13980/17125
13981/17125
13982/17125
13983/17125
13984/17125
13985/17125
13986/17125
13987/17125
13988/17125
13989/17125
13990/17125
13991/17125
13992/17125
13993/17125
13994/17125
13995/17125
13996/17125
13997/17125
13998/17125
13999/17125
14000/17125
14001/17125
14002/17125
14003/17125
14004/17125
14005/17125
14006/17125
14007/17125
14008/17125
14009/17125
14010/17125
14011/17125
14012/17125
14013/17125
14014/17125
14015/17125
14016/17125
14017/17125
14018/17125
14019/17125
14020/17125
14021/17125
14022/17125
14023/17125
14024/17125
14025/17125
14026/17125
14027/17125
14028/17125
14029/17125
14030/17125
14031/17125
14032/17125
14033/17125
1403

14637/17125
14638/17125
14639/17125
14640/17125
14641/17125
14642/17125
14643/17125
14644/17125
14645/17125
14646/17125
14647/17125
14648/17125
14649/17125
14650/17125
14651/17125
14652/17125
14653/17125
14654/17125
14655/17125
14656/17125
14657/17125
14658/17125
14659/17125
14660/17125
14661/17125
14662/17125
14663/17125
14664/17125
14665/17125
14666/17125
14667/17125
14668/17125
14669/17125
14670/17125
14671/17125
14672/17125
14673/17125
14674/17125
14675/17125
14676/17125
14677/17125
14678/17125
14679/17125
14680/17125
14681/17125
14682/17125
14683/17125
14684/17125
14685/17125
14686/17125
14687/17125
14688/17125
14689/17125
14690/17125
14691/17125
14692/17125
14693/17125
14694/17125
14695/17125
14696/17125
14697/17125
14698/17125
14699/17125
14700/17125
14701/17125
14702/17125
14703/17125
14704/17125
14705/17125
14706/17125
14707/17125
14708/17125
14709/17125
14710/17125
14711/17125
14712/17125
14713/17125
14714/17125
14715/17125
14716/17125
14717/17125
14718/17125
14719/17125
1472

15320/17125
15321/17125
15322/17125
15323/17125
15324/17125
15325/17125
15326/17125
15327/17125
15328/17125
15329/17125
15330/17125
15331/17125
15332/17125
15333/17125
15334/17125
15335/17125
15336/17125
15337/17125
15338/17125
15339/17125
15340/17125
15341/17125
15342/17125
15343/17125
15344/17125
15345/17125
15346/17125
15347/17125
15348/17125
15349/17125
15350/17125
15351/17125
15352/17125
15353/17125
15354/17125
15355/17125
15356/17125
15357/17125
15358/17125
15359/17125
15360/17125
15361/17125
15362/17125
15363/17125
15364/17125
15365/17125
15366/17125
15367/17125
15368/17125
15369/17125
15370/17125
15371/17125
15372/17125
15373/17125
15374/17125
15375/17125
15376/17125
15377/17125
15378/17125
15379/17125
15380/17125
15381/17125
15382/17125
15383/17125
15384/17125
15385/17125
15386/17125
15387/17125
15388/17125
15389/17125
15390/17125
15391/17125
15392/17125
15393/17125
15394/17125
15395/17125
15396/17125
15397/17125
15398/17125
15399/17125
15400/17125
15401/17125
15402/17125
1540

16009/17125
16010/17125
16011/17125
16012/17125
16013/17125
16014/17125
16015/17125
16016/17125
16017/17125
16018/17125
16019/17125
16020/17125
16021/17125
16022/17125
16023/17125
16024/17125
16025/17125
16026/17125
16027/17125
16028/17125
16029/17125
16030/17125
16031/17125
16032/17125
16033/17125
16034/17125
16035/17125
16036/17125
16037/17125
16038/17125
16039/17125
16040/17125
16041/17125
16042/17125
16043/17125
16044/17125
16045/17125
16046/17125
16047/17125
16048/17125
16049/17125
16050/17125
16051/17125
16052/17125
16053/17125
16054/17125
16055/17125
16056/17125
16057/17125
16058/17125
16059/17125
16060/17125
16061/17125
16062/17125
16063/17125
16064/17125
16065/17125
16066/17125
16067/17125
16068/17125
16069/17125
16070/17125
16071/17125
16072/17125
16073/17125
16074/17125
16075/17125
16076/17125
16077/17125
16078/17125
16079/17125
16080/17125
16081/17125
16082/17125
16083/17125
16084/17125
16085/17125
16086/17125
16087/17125
16088/17125
16089/17125
16090/17125
16091/17125
1609

16696/17125
16697/17125
16698/17125
16699/17125
16700/17125
16701/17125
16702/17125
16703/17125
16704/17125
16705/17125
16706/17125
16707/17125
16708/17125
16709/17125
16710/17125
16711/17125
16712/17125
16713/17125
16714/17125
16715/17125
16716/17125
16717/17125
16718/17125
16719/17125
16720/17125
16721/17125
16722/17125
16723/17125
16724/17125
16725/17125
16726/17125
16727/17125
16728/17125
16729/17125
16730/17125
16731/17125
16732/17125
16733/17125
16734/17125
16735/17125
16736/17125
16737/17125
16738/17125
16739/17125
16740/17125
16741/17125
16742/17125
16743/17125
16744/17125
16745/17125
16746/17125
16747/17125
16748/17125
16749/17125
16750/17125
16751/17125
16752/17125
16753/17125
16754/17125
16755/17125
16756/17125
16757/17125
16758/17125
16759/17125
16760/17125
16761/17125
16762/17125
16763/17125
16764/17125
16765/17125
16766/17125
16767/17125
16768/17125
16769/17125
16770/17125
16771/17125
16772/17125
16773/17125
16774/17125
16775/17125
16776/17125
16777/17125
16778/17125
1677

In [23]:
import os
import nmslib
import numpy as np
from towhee import AutoPipes
import time

# PIL (pillow) allows rendering of images in Python
from PIL import Image

index = nmslib.init(method = 'hnsw', space = 'l2')
p = AutoPipes.pipeline('image-embedding')

path = './JPEGImages'
dir_list = os.listdir(path)

index.loadIndex('imageDB', True)

# vectorize a form of an image through its URL to be searched
# this URL can be changed to any image you would like to search
query = p('https://cf.ltkcdn.net/aquariums/fish-names/images/orig/324592-1600x1066-goldfish-swimming-in-fish-tank.jpg').get()

# find 5 nearest neighbors
k = 5

ids, distances = index.knnQuery(query, k = k)

print("Query Point:", query)
print("Nearest Neighbors:")
for i, idx in enumerate(ids):
    print("Index:", idx, "Distance:", distances[i])
    image = Image.open(f'{path}/{dir_list[idx]}')
    image.show()
    time.sleep(1)

Query Point: [array([0.        , 0.        , 0.        , ..., 0.        , 0.00684197,
       0.        ], dtype=float32)]
Nearest Neighbors:
Index: 16220 Distance: 4.5640078
Index: 10817 Distance: 5.899211
Index: 214 Distance: 7.199253
Index: 10472 Distance: 8.874672
Index: 13712 Distance: 9.785667
